# Packages

In [1]:
from tensorflow import keras
from PIL import Image
from torchvision import models, transforms
import torch.nn as nn
import torch.optim as optim
import torch
import random
from torchvision import models, transforms
from numpy.linalg import norm
import numpy as np
import math

# Load Features

In [4]:
root = '../../'

X_train = np.load( root + "Features/train_features_vgg16_cifar10.npy" ) # Shape = (50000, 4096)
y_train = np.load( root + "Features/train_labels_vgg16_cifar10.npy" ) # Shape = (50000,)

X_test = np.load( root + "Features/test_features_vgg16_cifar10.npy" ) # Shape = (10000, 4096)
y_test = np.load( root + "Features/test_labels_vgg16_cifar10.npy" ) # Shape = (10000,)

# Setup Model

In [5]:
class CustomNN(nn.Module):
    def __init__(self):
        super(CustomNN, self).__init__()
        self.fc_layers = nn.Sequential(
            nn.Linear(4096, 256),  # First fully connected layer
            nn.ReLU(),
            nn.Linear(256, 32)    # Second fully connected layer to reduce to 4000
        )

        # Initialize weights and biases from gaussian distribution
        for layer in self.fc_layers:
            if isinstance(layer, nn.Linear):
                nn.init.normal_(layer.weight, mean=0.0, std=0.01)  # Initialize weights
                nn.init.normal_(layer.bias, mean=0.0, std=0.01)    # Initialize biases

    def forward(self, x):
        return self.fc_layers(x)

In [6]:
model = CustomNN()

# Define Custom Loss Function

In [12]:
import torch.nn as nn

class CustomLoss(nn.Module):
    def __init__(self, eta = 0.25):
        super(CustomLoss, self).__init__()
        self.eta = eta

    def forward(self, outputs, targets):
        #S calculation
        print("S Start")
        targets = torch.tensor(targets)
        S = (targets[:, None] == targets).float() # Shape = (sample_size, sample_size)
        print("S Slut")


        #U calculation
        U = outputs

        #Calculate Theta
        print("Theta Start")
        dot_product_matrix = torch.matmul(U, U.T)
        dot_product_matrix # (sample_size, sample_size) Shape
        Theta = 1/2 * dot_product_matrix
        print("Theta Slut")


        #Calculate hash codes
        B = torch.sign(U) # Shape = (sample_size, hash_length)


        loss = - torch.sum(S * Theta - torch.log(1 + torch.exp(Theta))) + self.eta * torch.sum(torch.norm(B - U, dim = 1).pow(2))
        print(loss) # Should give tensor(loss, grad_fn) NO CLUE HVAD GRAD_FN ER, NOK HVORDAN GRADIENT SKAL UDREGNES
        return loss.mean()


# Initialize Custom_loss And Optimizer

In [13]:
custom_loss = CustomLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Learn Model Parameters

In [ ]:
num_epochs = 1000

for epoch in range(num_epochs):
    # Zero gradients from the previous iteration
    optimizer.zero_grad()
    
    outputs = model(torch.tensor(X_train))

    # Forward pass
    loss = custom_loss(outputs, y_train)
    
    
    # Backward pass to compute gradients
    loss.backward()
    
    # Update parameters using optimizer
    optimizer.step()
    
    # Optionally, print loss for monitoring
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")

S Start
S Slut
Theta Start
Theta Slut
